# mergeSchema example

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[1]").appName("mrpowers").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/16 18:53:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.createDataFrame([(1, "a"), (2, "b")], ["col_a", "col_b"])

In [5]:
df.show()

+-----+-----+
|col_a|col_b|
+-----+-----+
|    1|    a|
|    2|    b|
+-----+-----+



In [6]:
df.write.format("parquet").save("tmp/parquet_table")

In [7]:
df = spark.createDataFrame([("x", "fun"), ("y", "bye")], ["col_b", "col_c"])

In [8]:
df.write.format("parquet").mode("append").save("tmp/parquet_table")

## "Regular read" doesn't give you desired result

In [9]:
spark.read.format("parquet").load("tmp/parquet_table").show()

+-----+-----+
|col_a|col_b|
+-----+-----+
|    1|    a|
|    2|    b|
| null|    x|
| null|    y|
+-----+-----+



## You have to set mergeSchema to properly read the data

In [10]:
spark.read.format("parquet").option("mergeSchema", True).load(
    "tmp/parquet_table"
).show()

+-----+-----+-----+
|col_a|col_b|col_c|
+-----+-----+-----+
|    1|    a| null|
|    2|    b| null|
| null|    x|  fun|
| null|    y|  bye|
+-----+-----+-----+

